<a href="https://colab.research.google.com/github/ali1810/Deep_Chem/blob/main/model_solubility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
### Installing RDKIT 
!mamba install -c conda-forge rdkit

Streaming output truncated to the last 5000 lines.
xorg-renderproto         [] (00m:00s) Waiting...
xorg-xproto              [] (00m:00s) Waiting...
xorg-kbproto             [] (00m:00s) Waiting...
fontconfig               [] (00m:00s) Waiting...
cairo                    [] (00m:00s) Waiting...
libblas                  [] (00m:00s) Waiting...
jpeg                     [] (00m:00s) Waiting...
gettext                  [] (00m:00s) Waiting...
libwebp                  [] (00m:00s) Waiting...
python-dateutil          [] (00m:00s) Waiting...
libopenblas              [] (00m:00s)      6 MB /      9 MB (  4.58 MB/s)
typing_extensions        [] (00m:00s) Waiting...
numpy                    [] (00m:00s)      5 MB /      6 MB (  4.34 MB/s)
pillow                   [] (00m:00s) Waiting...
xorg-libx11              [] (00m:00s) Waiting...
cycler                   [] (00m:00s) Waiting...
greenlet                 [] (00m:00s) Waiting...
libgfortran-ng           [] (00m:00s) Waiting...
xorg-xextproto   

In [3]:
import keras as k
from keras.models import Sequential, Model
from keras.layers import Activation, BatchNormalization, Input, Dense
from keras.layers.core import Dense
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras import losses
from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors, Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Fingerprints import FingerprintMols
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import argmax
from numpy import ndarray
import tensorflow as tf
import math
import re
import string
from collections import OrderedDict
import pandas as pd
import csv
from collections import defaultdict

In [4]:
"""DATA PREPROCESSING"""
with open('/content/drive/MyDrive/1K_unformated_unlabeled_SMILES_data.txt') as my_file:
    dataset = my_file.readlines()

with open('/content/drive/MyDrive/test_unformated_unlabeled_SMILES_data.txt') as my_file:
    test_dataset = my_file.readlines()


In [5]:
#Formating data into array of arrays
array_of_array = []

for line in dataset:
        formated_data = line.split()
        array_of_array.append(formated_data)

test_array_of_array = []

for test_line in test_dataset:
        test_formated_data = line.split()
        test_array_of_array.append(test_formated_data)



In [6]:
#Extracting the SMILES strings from excel dataset

formated_SMILE_array = []

c_reader = csv.reader(open('/content/drive/MyDrive/1K_unformated_unlabeled_SMILES_data.txt', 'r'), delimiter=';')
col_2 = [x[0] for x in c_reader]
for string in col_2:
        head, sep, tail = string.partition('\t')
        SMILE = tail
        formated_SMILE_array.append(SMILE)

test_formated_SMILE_array = []

test_c_reader = csv.reader(open('/content/drive/MyDrive/test_unformated_unlabeled_SMILES_data.txt', 'r'), delimiter=';')
test_col_2 = [x[0] for x in test_c_reader]
for test_string in test_col_2:
        head, sep, tail = test_string.partition('\t')
        test_SMILE = tail
        test_formated_SMILE_array.append(SMILE)

In [7]:
#Extracting the name of the Molecule
for data_array in array_of_array:
        for info in data_array:
                name_ind_position = data_array[2]

for test_data_array in test_array_of_array:
        for test_info in test_data_array:
                test_name_ind_position = test_data_array[2]


In [8]:
#Extracting the solubility levels
solubility_array = []

for data_array in array_of_array:
        solubility_ind_position = data_array[3]
        solubility_array.append(solubility_ind_position)

test_solubility_array = []
for test_data_array in test_array_of_array:
        test_solubility_ind_position = test_data_array[3]
        test_solubility_array.append(test_solubility_ind_position)


In [104]:
#arr = solubility_array.astype('float64')

solubility_array_new = np.array(solubility_array).astype('float64')
test_solubility_array_new= np.array(test_solubility_array).astype('float64')

print(fingerprints_array_new[0]) 
print(solubility_array_new[0])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-3.18


In [106]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fingerprints_array_new, solubility_array_new, test_size = 0.1, random_state = 0)
print(len(X_train))
print(len(y_train)) 

900
900


In [113]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 2000, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=2000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [114]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


[[-4.6  -4.4 ]
 [-5.21 -6.35]
 [-0.82  0.52]
 [-2.86 -5.46]
 [-2.09 -1.61]
 [-4.26 -5.03]
 [-3.89 -3.01]
 [-3.68 -1.24]
 [-2.41 -0.83]
 [-2.75 -2.44]
 [-4.09 -3.59]
 [-2.89 -3.67]
 [-3.62 -1.04]
 [-1.2  -0.5 ]
 [-1.4  -1.64]
 [-4.5  -3.7 ]
 [-0.76 -1.6 ]
 [-5.31 -4.88]
 [-2.54 -3.23]
 [-7.47 -9.03]
 [-3.04 -2.8 ]
 [-2.5  -1.87]
 [-1.02  0.22]
 [-2.12 -3.31]
 [-3.78 -3.54]
 [-3.34 -2.98]
 [-3.41 -4.  ]
 [-2.14 -2.6 ]
 [-3.2  -5.54]
 [-3.42 -2.64]
 [-2.07  1.07]
 [-0.95 -1.01]
 [-1.15 -1.61]
 [-4.13 -4.79]
 [-3.25 -3.69]
 [-3.09 -4.89]
 [-2.28 -2.17]
 [-0.56 -0.22]
 [-1.44 -0.1 ]
 [-1.26 -1.38]
 [-0.56 -0.2 ]
 [-3.47 -3.66]
 [-3.85 -2.82]
 [-2.1  -1.6 ]
 [-0.6  -1.38]
 [-3.96 -8.4 ]
 [-0.34  1.06]
 [-2.19 -1.36]
 [ 0.18  0.38]
 [-2.87 -0.62]
 [-3.51 -2.21]
 [-0.3   0.32]
 [-0.62 -1.27]
 [-0.49  0.93]
 [-5.68 -8.06]
 [-2.87 -2.07]
 [-0.35  0.  ]
 [-2.86 -0.88]
 [-2.74 -4.38]
 [-4.14 -1.06]
 [-1.82 -4.19]
 [-1.65 -2.63]
 [-5.17 -6.29]
 [-6.68 -8.56]
 [-3.86 -3.77]
 [-6.39 -6.51]
 [-1.39 -0

In [115]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6238588937971519

In [88]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

# Training the SVR model on the Training set
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

# Evaluating the Model Performance
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

ValueError: ignored

In [58]:
#test_solubility_array_new.dtype
fingerprints_array_new= np.array(fingerprints_array).astype('float64')
test_fingerprints_array_new = np.array(test_fingerprints_array).astype('float64')

fingerprints_array_new.dtype


dtype('float64')

In [10]:
#Turning SMILES into Explicit Bit Vectors (RDKit prefered format)
mols = [Chem.rdmolfiles.MolFromSmiles(SMILES_string) for SMILES_string in formated_SMILE_array]

test_mols = [Chem.rdmolfiles.MolFromSmiles(test_SMILES_string) for test_SMILES_string in test_formated_SMILE_array]

#Convert training molecules into training fingerprints
bi = {}
fingerprints = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m, radius=2, bitInfo= bi, nBits=256) for m in mols]

test_bi = {}
test_fingerprints = [rdMolDescriptors.GetMorganFingerprintAsBitVect(test_m, radius=2, bitInfo= bi, nBits=256) for test_m in test_mols]

In [77]:
#Convert training fingerprints into binary, and put all training binaries into arrays
fingerprints_array = []
for fingerprint in fingerprints:
        array = np.zeros((1,), dtype= int)
        DataStructs.ConvertToNumpyArray(fingerprint, array)
        fingerprints_array.append(array)

test_fingerprints_array = []
for test_fingerprint in test_fingerprints:
        test_array = np.zeros((1,), dtype= int)
        DataStructs.ConvertToNumpyArray(test_fingerprint, test_array)
        test_fingerprints_array.append(test_array)

#print (fingerprints_array, solubility_array)

In [50]:
"""NEURAL NETWORK"""
#The neural network model
model = Sequential([
    Dense(256, input_shape=(256,), activation= "relu"),
    Dense(128, activation= "tanh"),
    Dense(64, activation= "tanh"),
    Dense(34, activation= "tanh"),
    Dense(16, activation= "tanh"),
    BatchNormalization(axis=1),
    Dense(1, activation= "tanh")
])

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 34)                2210      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                560       
_________________________________________________________________
batch_normalization (BatchNo (None, 16)                64        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1

In [63]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators =10, random_state = 0)
regressor.fit(fingerprints_array_new, solubility_array_new)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [64]:
y_pred = regressor.predict(test_fingerprints_array)
#np.set_printoptions(precision=2)
#print(y_pred)
#print(test_solubility_array_new)
print(np.concatenate((y_pred.reshape(len(y_pred),1), test_solubility_array_new.reshape(len(test_solubility_array_new),1)),1))

[[-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]
 [-6.23 -7.32]]


In [47]:
from sklearn.metrics import r2_score
r2_score(test_solubility_array_new, y_pred)

-3.792924684436885e+29

In [55]:
fingerprints_array.dtype

AttributeError: ignored

In [118]:
#Compiling the model
model.compile(loss=None, optimizer=Adam(learning_rate=0.0005), metrics=["accuracy"])


In [ ]:
x=np.array(fingerprints_array)

In [ ]:
y_new = y.astype(np.float)

In [122]:
model.fit(X_train, y_train, epochs= 200, shuffle=True, verbose=1)


Epoch 1/200


ValueError: ignored

In [121]:
#Training the model
model.fit(np.array(fingerprints_array), np.array(solubility_array), validation_split=0.1, batch_size=25, epochs= 200, shuffle=True, verbose=1)


Epoch 1/200


ValueError: ignored

In [123]:
model.fit(np.array(fingerprints_array_new), np.array(solubility_array_new), validation_split=0.1, batch_size=25, epochs= 200, shuffle=True, verbose=1)


Epoch 1/200


ValueError: ignored